### View Dataset

In [8]:
import pandas

# Read the dataset into a data table using Pandas
data_table = pandas.read_csv("housesale.csv")

print(data_table.shape)
print(list(data_table.columns))
data_table.head()


(17030, 15)
['Township', 'PIN', 'Class', 'Address', 'Grantor', 'Grantee', 'Sale Type', 'SaleDate', 'Sale Price', 'Notes', 'Latitude', 'Longitude', 'Year', 'Month', 'Quarter']


,Township,PIN,Class,Address,Grantor,Grantee,Sale Type,SaleDate,Sale Price,Notes,Latitude,Longitude,Year,Month,Quarter
0,BURRITT,10-03-300-005,RURAL IMPROVED,CEMETERY RD,MC DERMOTT STEVEN,KINGSBURY SEAN K,MULTI,9/23/15,"193,000",WARRANTY DEED,42.363380,-89.205514,2015,9,3
1,BURRITT,10-03-300-006,RURAL IMPROVED,9473 CEMETERY RD,NOWICKI JENNIFER,KINGSBURY SEAN K,MULTI,9/23/15,"193,000",WARRANTY DEED,42.363320,-89.223087,2015,9,3
2,BURRITT,10-05-400-007,RURAL LAND,CHRISTIAN RD,MC GLONE ANN L TR,OLIVER SEAN,MULTI,8/4/15,"75,000","TRUSTEE'S DEED, NOT\rEXPOSED TO OPEN MRKT",42.301555,-89.147751,2015,8,3
3,BURRITT,10-08-200-003,RURAL IMPROVED,10621 CEMETERY RD,SULLIVAN JAMES E TRUST,OLIVER SEAN,MULTI,8/4/15,"75,000","TRUSTEE'S DEED, NOT\rEXPOSED TO OPEN MRKT",42.362106,-89.245893,2015,8,3
4,BURRITT,10-08-200-018,RURAL IMPROVED,10650 CEMETERY RD,JOHNSON DEBRA L,COMBS SUSAN,PASMT,11/23/15,"175,000",NaN,42.360278,-89.245793,2015,11,4


### Data exploration and visulization